In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 불러오기 - v3.2

In [2]:
%pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 2.6 MB/s eta 0:00:00


In [3]:
%pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.0 MB/s eta 0:00:00


In [17]:
import pandas as pd
import numpy as np
import optuna
import category_encoders as ce
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, SelectPercentile, SelectFromModel

In [5]:
DATA_PATH = "/content/drive/MyDrive/멋쟁이사자처럼/Final_Project/dataset/"
RESULT_PATH = "/content/drive/MyDrive/멋쟁이사자처럼/Final_Project/result/"
SEED = 42

In [6]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.2_1107.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.2_1107.csv")
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv")
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv")

train_ft.shape, test_ft.shape, train_target.shape, submit.shape

((14940, 1455), (12225, 1455), (14940, 2), (12225, 2))

## 결측치 처리 및 스케일링

In [7]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

In [8]:
# ID 제거
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 1454), (12225, 1454))

In [9]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

,0
주구매지점,4
주구매_대분류,22
주구매_중분류,238


In [10]:
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft["주구매지점"])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft["주구매지점"])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 1458), (12225, 1458))

In [11]:
enc = ce.count.CountEncoder()
train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft["주구매_대분류_cnt"] = enc.fit_transform(train_ft[["주구매_대분류"]])
test_ft["주구매_대분류_cnt"] = enc.transform(test_ft[["주구매_대분류"]])

train_ft.shape, test_ft.shape

((14940, 1460), (12225, 1460))

In [12]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 1457), (12225, 1457))

In [13]:
scaler = StandardScaler()

train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

,내점일수,구매주기,주말방문비율,평일방문비율,주말방문횟수,평일방문횟수,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,...,공휴일_대분류_영플라자_구매횟수,공휴일_대분류_잡화_구매횟수,공휴일_대분류_케주얼_구두_아동_구매횟수,공휴일_대분류_패션잡화_구매횟수,주구매지점_1,주구매지점_2,주구매지점_3,주구매지점_4,주구매_중분류_cnt,주구매_대분류_cnt
0,-0.369867,0.002987,0.257728,-0.257728,-0.230862,-0.390544,-1.029777,0.001191,0.838272,0.338186,...,-0.162079,-0.286379,-0.209907,-0.213001,1.654066,-0.623175,-0.612523,-0.468181,-0.741478,-1.315250
1,0.144110,-0.356452,-1.008554,1.008554,-0.619841,0.412809,0.323951,-0.390607,0.620171,-0.552996,...,2.389509,-0.286379,-0.209907,4.161199,-0.604571,1.604686,-0.612523,-0.468181,-0.448069,-0.626416
2,1.943028,-0.869935,0.036742,-0.036742,1.616788,1.926821,0.798943,-0.514333,-0.304527,-0.059266,...,-0.162079,-0.286379,-0.209907,-0.213001,-0.604571,-0.623175,1.632592,-0.468181,-0.869072,1.064726
3,3.793345,-1.023980,-0.080558,0.080558,3.172703,4.429574,0.420933,-0.327474,0.008592,-0.135636,...,10.044273,-0.286379,1.301424,-0.213001,1.654066,-0.623175,-0.612523,-0.468181,1.515861,-0.248364
4,0.452496,-0.613193,0.302875,-0.302875,0.838830,0.567300,-0.752532,1.707410,-0.130285,-0.821561,...,-0.162079,4.377687,4.324087,-0.213001,1.654066,-0.623175,-0.612523,-0.468181,-0.847712,1.064726


In [14]:
target = train_target["target"]

## cv 점수 확인해보기

In [16]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

params = {'n_estimators': 800,
 'learning_rate': 0.04036413044768581,
 'max_depth': 4,
 'min_child_weight': 7,
 'subsample': 0.7505214930635562,
 'colsample_bytree': 0.6290102054237857,
 'gamma': 0.648553153047272}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# Stratified K-Fold 교차 검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    model = XGBClassifier(**params)
    model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], verbose=False)

    # 모델 저장
    models.append(model)

    # 예측 및 F1 매크로 스코어 계산
    pred = model.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))

Mean F1 Macro Score: 0.7184388251001332


## Feature Selection

In [18]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [19]:
xgb = XGBClassifier(random_state=42) # 특성 선택을 하기 위한 모델
model = XGBClassifier(random_state=42) # cv 점수 확인

fs = SelectFromModel(xgb)
x = fs.fit_transform(train_ft, target)
scores = cross_val_score(model, x, target, cv=cv, scoring="f1_macro", n_jobs=-1)
scores.mean()

0.7089280932862698

In [20]:
best_cols = fs.get_feature_names_out()
len(best_cols)

552

## Decision Tree

In [21]:
def objective(trial):
    hp = {
        "criterion": trial.suggest_categorical("criterion", ["gini", "entropy"]),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20)
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    model = DecisionTreeClassifier(**hp, random_state=SEED)
    score = cross_val_score(model, train_ft[best_cols], target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
    return score

sampler = optuna.samplers.TPESampler(seed=SEED)

study = optuna.create_study(
    direction="maximize",
    sampler=sampler
)

study.optimize(objective, n_trials=100)

[I 2024-11-08 00:39:13,512] A new study created in memory with name: no-name-6461b71d-2dca-4632-96c1-3cd20bcd3df5
[I 2024-11-08 00:39:33,042] Trial 0 finished with value: 0.631642881942922 and parameters: {'criterion': 'entropy', 'max_depth': 15, 'min_samples_split': 13, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.631642881942922.
[I 2024-11-08 00:39:46,392] Trial 1 finished with value: 0.6300912127391963 and parameters: {'criterion': 'gini', 'max_depth': 18, 'min_samples_split': 13, 'min_samples_leaf': 15}. Best is trial 0 with value: 0.631642881942922.
[I 2024-11-08 00:40:03,483] Trial 2 finished with value: 0.6291253987128721 and parameters: {'criterion': 'entropy', 'max_depth': 17, 'min_samples_split': 6, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.631642881942922.
[I 2024-11-08 00:40:17,414] Trial 3 finished with value: 0.6436516604041864 and parameters: {'criterion': 'entropy', 'max_depth': 11, 'min_samples_split': 10, 'min_samples_leaf': 6}. Best is trial 3 w

In [22]:
# 최적의 하이퍼파라미터 출력
print("Best hyperparameters:", study.best_params)
print("Best F1 Score:", study.best_value)

Best hyperparameters: {'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 12, 'min_samples_leaf': 17}
Best F1 Score: 0.6648316741189519


In [23]:
# 모델 학습
model = DecisionTreeClassifier(random_state=SEED, **study.best_params)
model.fit(train_ft[best_cols],target)

pred = model.predict(test_ft[best_cols])

submit["target"] = pred
submit.to_csv(f"{RESULT_PATH}6_v3.2_DecisionTree.csv",index=False)

## RandomForest

In [27]:
def objective(trial):
    hp = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 300, step=50),
        "criterion": trial.suggest_categorical("criterion", ["gini", "entropy"]),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None])
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    model = RandomForestClassifier(**hp, random_state=SEED)
    score = cross_val_score(model, train_ft[best_cols], target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
    return score

sampler = optuna.samplers.TPESampler(seed=SEED)

study = optuna.create_study(
    direction="maximize",
    sampler=sampler
)

study.optimize(objective, n_trials=50)

[I 2024-11-08 07:09:27,779] A new study created in memory with name: no-name-ea0d603b-9bf7-4343-8136-73330d9c941e
[I 2024-11-08 07:09:56,392] Trial 0 finished with value: 0.6857064240262093 and parameters: {'n_estimators': 150, 'criterion': 'gini', 'max_depth': 12, 'min_samples_split': 4, 'min_samples_leaf': 4, 'max_features': 'log2'}. Best is trial 0 with value: 0.6857064240262093.
[W 2024-11-08 07:11:35,307] Trial 1 failed with parameters: {'n_estimators': 250, 'criterion': 'entropy', 'max_depth': 17, 'min_samples_split': 6, 'min_samples_leaf': 4, 'max_features': None} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-27-2506c490aa32>", line 13, in objective
    score = cross_val_score(model, train_ft[best_cols], target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
  File "/usr/local/lib/pyth

KeyboardInterrupt: 

In [ ]:
# 최적의 하이퍼파라미터 출력
print("Best hyperparameters:", study.best_params)
print("Best F1 Score:", study.best_value)

In [ ]:
# 모델 학습
model = RandomForestClassifier(random_state=SEED, **study.best_params)
model.fit(train_ft[best_cols],target)

pred = model.predict(test_ft[best_cols])

submit["target"] = pred
submit.to_csv(f"{RESULT_PATH}6_v3.2_RF.csv",index=False)

## Histgbm

In [ ]:
# hist = HistGradientBoostingClassifier(random_state=SEED) # 특성 선택을 하기 위한 모델
# model = LogisticRegression(random_state=SEED)

# fs_hist = SelectFromModel(hist)
# x = fs_hist.fit_transform(train_ft, target)
# scores = cross_val_score(model, x, target, cv=cv, scoring="f1_macro", n_jobs=-1)
# scores.mean()

In [ ]:
# best_cols = fs_hist.get_feature_names_out()
# len(best_cols)

In [25]:
def objective(trial):
    hp = {
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),  # Learning rate
        "max_iter": trial.suggest_int("max_iter", 100, 500, step=50),  # Number of boosting iterations
        "max_depth": trial.suggest_int("max_depth", 3, 20),  # Maximum depth of each tree
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),  # Minimum samples at a leaf node
        "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 10, 100, step=10)  # Max leaf nodes
    }

    cv = KFold(5, shuffle=True, random_state=SEED)
    model = HistGradientBoostingClassifier(**hp, random_state=SEED)
    score = cross_val_score(model, train_ft, target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
    return score

sampler = optuna.samplers.TPESampler(seed=SEED)

study = optuna.create_study(
    direction="maximize",
    sampler=sampler
)

study.optimize(objective, n_trials=100)

[I 2024-11-08 01:09:59,897] A new study created in memory with name: no-name-07ede90e-e05a-4b23-b5a0-af19c71ea043
[I 2024-11-08 01:13:23,808] Trial 0 finished with value: 0.716034424084919 and parameters: {'learning_rate': 0.030710573677773714, 'max_iter': 500, 'max_depth': 16, 'min_samples_leaf': 12, 'max_leaf_nodes': 20}. Best is trial 0 with value: 0.716034424084919.
[I 2024-11-08 01:17:24,953] Trial 1 finished with value: 0.6972042506519933 and parameters: {'learning_rate': 0.015957084694148364, 'max_iter': 100, 'max_depth': 18, 'min_samples_leaf': 13, 'max_leaf_nodes': 80}. Best is trial 0 with value: 0.716034424084919.
[I 2024-11-08 01:25:14,938] Trial 2 finished with value: 0.7132036726186279 and parameters: {'learning_rate': 0.010636066512540286, 'max_iter': 500, 'max_depth': 17, 'min_samples_leaf': 5, 'max_leaf_nodes': 20}. Best is trial 0 with value: 0.716034424084919.
[I 2024-11-08 01:29:35,549] Trial 3 finished with value: 0.7094077778012592 and parameters: {'learning_rate'

KeyboardInterrupt: 

In [21]:
hist_best_params = {'learning_rate': 0.034154760958502,
                    'max_iter': 300,
                    'max_depth': 12,
                    'min_samples_leaf': 16,
                    'max_leaf_nodes': 40}

# best_value = 0.7193354225222076

In [ ]:
# 최적의 하이퍼파라미터 출력
print("Best hyperparameters:", study.best_params)
print("Best F1 Score:", study.best_value)

In [22]:
# 모델 학습
model = HistGradientBoostingClassifier(random_state=SEED, **hist_best_params)
model.fit(train_ft[best_cols],target)

pred = model.predict(test_ft[best_cols])

submit["target"] = pred
submit.to_csv(f"{RESULT_PATH}6_v3.2_HistGBM.csv",index=False)

# 데이터 불러오기 - v3.2 피처 삭제 X

In [23]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.2_피처삭제X_1107.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.2_피처삭제X_1107.csv")
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv")
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv")

train_ft.shape, test_ft.shape, train_target.shape, submit.shape

((14940, 1470), (12225, 1470), (14940, 2), (12225, 2))

## 결측치 처리 및 스케일링

In [24]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

In [25]:
# ID 제거
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 1469), (12225, 1469))

In [26]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

,0
주구매지점,4
주구매_대분류,22
주구매_중분류,238


In [27]:
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft["주구매지점"])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft["주구매지점"])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 1473), (12225, 1473))

In [28]:
enc = ce.count.CountEncoder()
train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft["주구매_대분류_cnt"] = enc.fit_transform(train_ft[["주구매_대분류"]])
test_ft["주구매_대분류_cnt"] = enc.transform(test_ft[["주구매_대분류"]])

train_ft.shape, test_ft.shape

((14940, 1475), (12225, 1475))

In [29]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 1472), (12225, 1472))

In [30]:
scaler = StandardScaler()

train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

,내점일수,구매주기,주말방문비율,평일방문비율,주말방문횟수,평일방문횟수,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,...,공휴일_대분류_영플라자_구매횟수,공휴일_대분류_잡화_구매횟수,공휴일_대분류_케주얼_구두_아동_구매횟수,공휴일_대분류_패션잡화_구매횟수,주구매지점_1,주구매지점_2,주구매지점_3,주구매지점_4,주구매_중분류_cnt,주구매_대분류_cnt
0,-0.369867,0.002987,0.257728,-0.257728,-0.230862,-0.390544,-1.029777,0.001191,0.838272,0.338186,...,-0.162079,-0.286379,-0.209907,-0.213001,1.654066,-0.623175,-0.612523,-0.468181,-0.741478,-1.315250
1,0.144110,-0.356452,-1.008554,1.008554,-0.619841,0.412809,0.323951,-0.390607,0.620171,-0.552996,...,2.389509,-0.286379,-0.209907,4.161199,-0.604571,1.604686,-0.612523,-0.468181,-0.448069,-0.626416
2,1.943028,-0.869935,0.036742,-0.036742,1.616788,1.926821,0.798943,-0.514333,-0.304527,-0.059266,...,-0.162079,-0.286379,-0.209907,-0.213001,-0.604571,-0.623175,1.632592,-0.468181,-0.869072,1.064726
3,3.793345,-1.023980,-0.080558,0.080558,3.172703,4.429574,0.420933,-0.327474,0.008592,-0.135636,...,10.044273,-0.286379,1.301424,-0.213001,1.654066,-0.623175,-0.612523,-0.468181,1.515861,-0.248364
4,0.452496,-0.613193,0.302875,-0.302875,0.838830,0.567300,-0.752532,1.707410,-0.130285,-0.821561,...,-0.162079,4.377687,4.324087,-0.213001,1.654066,-0.623175,-0.612523,-0.468181,-0.847712,1.064726


In [31]:
target = train_target["target"]

## cv 점수 확인해보기

In [32]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

params = {'n_estimators': 800,
 'learning_rate': 0.04036413044768581,
 'max_depth': 4,
 'min_child_weight': 7,
 'subsample': 0.7505214930635562,
 'colsample_bytree': 0.6290102054237857,
 'gamma': 0.648553153047272}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# Stratified K-Fold 교차 검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    model = XGBClassifier(**params)
    model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], verbose=False)

    # 모델 저장
    models.append(model)

    # 예측 및 F1 매크로 스코어 계산
    pred = model.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))

Mean F1 Macro Score: 0.7194134085258163


## Feature Selection

In [33]:
xgb = XGBClassifier(random_state=42) # 특성 선택을 하기 위한 모델
model = LogisticRegression(random_state=42)

fs = SelectFromModel(xgb)
x = fs.fit_transform(train_ft, target)
scores = cross_val_score(model, x, target, cv=cv, scoring="f1_macro", n_jobs=-1)
scores.mean()

0.7132655307053797

In [34]:
best_cols = fs.get_feature_names_out()
len(best_cols)

552

## Decision Tree

In [39]:
def objective(trial):
    hp = {
        "criterion": trial.suggest_categorical("criterion", ["gini", "entropy"]),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20)
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    model = DecisionTreeClassifier(**hp, random_state=SEED)
    score = cross_val_score(model, train_ft[best_cols], target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
    return score

sampler = optuna.samplers.TPESampler(seed=SEED)

study = optuna.create_study(
    direction="maximize",
    sampler=sampler
)

study.optimize(objective, n_trials=100)

[I 2024-11-08 07:13:29,291] A new study created in memory with name: no-name-4807a470-6626-4764-a5fd-eed0b93a7ece
[I 2024-11-08 07:13:56,294] Trial 0 finished with value: 0.631642881942922 and parameters: {'criterion': 'entropy', 'max_depth': 15, 'min_samples_split': 13, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.631642881942922.
[I 2024-11-08 07:14:16,952] Trial 1 finished with value: 0.6300912127391963 and parameters: {'criterion': 'gini', 'max_depth': 18, 'min_samples_split': 13, 'min_samples_leaf': 15}. Best is trial 0 with value: 0.631642881942922.
[I 2024-11-08 07:14:44,107] Trial 2 finished with value: 0.6291253987128721 and parameters: {'criterion': 'entropy', 'max_depth': 17, 'min_samples_split': 6, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.631642881942922.
[I 2024-11-08 07:14:58,174] Trial 3 finished with value: 0.6436516604041864 and parameters: {'criterion': 'entropy', 'max_depth': 11, 'min_samples_split': 10, 'min_samples_leaf': 6}. Best is trial 3 w

In [40]:
# 최적의 하이퍼파라미터 출력
print("Best hyperparameters:", study.best_params)
print("Best F1 Score:", study.best_value)

Best hyperparameters: {'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 12, 'min_samples_leaf': 17}
Best F1 Score: 0.6648316741189519


In [41]:
# 모델 학습
model = DecisionTreeClassifier(random_state=SEED, **study.best_params)
model.fit(train_ft[best_cols],target)

pred = model.predict(test_ft[best_cols])

submit["target"] = pred
submit.to_csv(f"{RESULT_PATH}6_v3.2_피처삭제X_DecisionTree.csv",index=False)

## RandomForest

In [42]:
def objective(trial):
    hp = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 300, step=50),
        "criterion": trial.suggest_categorical("criterion", ["gini", "entropy"]),
        "max_depth": trial.suggest_int("max_depth", 1, 20),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None])
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    model = RandomForestClassifier(**hp, random_state=SEED)
    score = cross_val_score(model, train_ft[best_cols], target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
    return score

sampler = optuna.samplers.TPESampler(seed=SEED)

study = optuna.create_study(
    direction="maximize",
    sampler=sampler
)

study.optimize(objective, n_trials=50)

[I 2024-11-08 07:29:48,210] A new study created in memory with name: no-name-e6de6178-7d06-4ec6-98b9-79ad0a078dd8
[I 2024-11-08 07:30:10,296] Trial 0 finished with value: 0.6857064240262093 and parameters: {'n_estimators': 150, 'criterion': 'gini', 'max_depth': 12, 'min_samples_split': 4, 'min_samples_leaf': 4, 'max_features': 'log2'}. Best is trial 0 with value: 0.6857064240262093.
[W 2024-11-08 07:50:17,353] Trial 1 failed with parameters: {'n_estimators': 250, 'criterion': 'entropy', 'max_depth': 17, 'min_samples_split': 6, 'min_samples_leaf': 4, 'max_features': None} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-42-ab1ca3ae4f76>", line 13, in objective
    score = cross_val_score(model, train_ft[best_cols], target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
  File "/usr/local/lib/pyth

KeyboardInterrupt: 

In [ ]:
# 최적의 하이퍼파라미터 출력
print("Best hyperparameters:", study.best_params)
print("Best F1 Score:", study.best_value)

In [ ]:
# 모델 학습
model = DecisionTreeClassifier(random_state=SEED, **study.best_params)
model.fit(train_ft[best_cols],target)

pred = model.predict(test_ft[best_cols])

submit["target"] = pred
submit.to_csv(f"{RESULT_PATH}6_v3.2_피처삭제X_RandomForest.csv",index=False)

## HistGBM

In [35]:
def objective(trial):
    hp = {
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),  # Learning rate
        "max_iter": trial.suggest_int("max_iter", 100, 500, step=50),  # Number of boosting iterations
        "max_depth": trial.suggest_int("max_depth", 3, 20),  # Maximum depth of each tree
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 20),  # Minimum samples at a leaf node
        "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 10, 100, step=10)  # Max leaf nodes
    }

    cv = KFold(5, shuffle=True, random_state=SEED)
    model = HistGradientBoostingClassifier(**hp, random_state=SEED)
    score = cross_val_score(model, train_ft, target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
    return score

sampler = optuna.samplers.TPESampler(seed=SEED)

study = optuna.create_study(
    direction="maximize",
    sampler=sampler
)

study.optimize(objective, n_trials=60)

[I 2024-11-08 08:41:24,866] A new study created in memory with name: no-name-25f4c0e3-15f3-4af6-9d7e-a7383f7c29aa
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[I 2024-11-08 08:45:14,145] Trial 0 finished with value: 0.716034424084919 and parameters: {'learning_rate': 0.030710573677773714, 'max_iter': 500, 'max_depth': 16, 'min_samples_leaf': 12, 'max_leaf_nodes': 20}. Best is trial 0 with value: 0.716034424084919.
[I 2024-11-08 08:49:58,289] Trial 1 finished with value: 0.6972042506519933 and parameters: {'learning_rate': 0.015957084694148364, 'max_iter': 100, 'max_depth': 18, 'min_samples_leaf': 13, 'max_leaf_nodes': 80}. Best is trial 0 with value: 0.716034424084919.
[I 2024-11-08 08:58:53,473] Trial 2 finished with value: 0.7132036726186279 and parameters: {'learning_rate': 0.0106360

KeyboardInterrupt: 

In [ ]:
# 최적의 하이퍼파라미터 출력
print("Best hyperparameters:", study.best_params)
print("Best F1 Score:", study.best_value)

In [ ]:
# 모델 학습
model = DecisionTreeClassifier(random_state=SEED, **study.best_params)
model.fit(train_ft[best_cols],target)

pred = model.predict(test_ft[best_cols])

submit["target"] = pred
submit.to_csv(f"{RESULT_PATH}6_v3.2_피처삭제X_HistGBM.csv",index=False)